In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import pickle
import json
from itertools import cycle
from multiprocessing import Pool

In [2]:
with open('../tools/key.json', encoding='utf-8') as file:
    key = json.loads(file.read())
    
reqs = requests.get('https://proxy.webshare.io/api/proxy/list/', headers=key)

proxy_list = reqs.json()['results']
proxy_list = list(map(lambda x: f"http://{x['username']}:{x['password']}@{x['proxy_address']}:{x['ports']['http']}/", proxy_list))
proxy_cycle = cycle(proxy_list)

In [3]:
# urls = {'novel':{'https://www.anime-planet.com/'}, 'done':set(), 'cur':set()}
with open('../data/urls.pkl','rb') as file:
    urls = pickle.load(file)

In [4]:
def saveUrls():
    with open('../data/urls.pkl','wb') as file:
        pickle.dump(urls, file)

In [5]:
def getCurrentPageUrls(url):
    cur_urls = set()
    
    reqs = requests.get(url, proxies={'http': next(proxy_cycle)})
    soup = BeautifulSoup(reqs.text, 'html.parser')

    for link in soup.find_all('a'):
        try:
            branch = link.get('href')
            if branch[0] == '/':
                cur_urls.add('https://www.anime-planet.com' + branch)
        except:
            pass
    
    if len(cur_urls) == 0:
        raise Exception('Scrape unsuccessful')
    
    
    return cur_urls

In [6]:
def getAllUrls():
    while len(urls['novel']) > 0:
        batch_urls = []
        for _ in range(16):
            pop_url = urls['novel'].pop()
            if pop_url not in urls['done']:
                batch_urls.append(pop_url)

        with Pool(16) as p:
            cur_urls = p.map(getCurrentPageUrls, batch_urls)

        cur_urls = set().union(*cur_urls)

        urls['done'] = urls['done'].union(batch_urls)
        print(len(urls['novel']), len(urls['done']), len(urls['done'])/len(urls['novel']))

        diff = cur_urls.difference(urls['done'])
        urls['novel'] = urls['novel'].union(diff)
        saveUrls()

In [7]:
getAllUrls()

415239 57814 0.1392306599331951
415333 57830 0.13923767194034667
415385 57846 0.13925875994559264
415463 57862 0.13927112643003106
415564 57878 0.13927577942266414
415627 57894 0.13929316430357028
415654 57910 0.1393226096705433
415675 57926 0.1393540626691526
415756 57942 0.1393653970117088
415796 57958 0.13939047032679486
415855 57974 0.13940916906133147


KeyboardInterrupt: 